In [1]:
pip install opencv-python-headless numpy scikit-image


Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
import os
from sklearn.cluster import KMeans
import json

In [3]:
def extract_color_histogram(frame, bins=64):
    """Trích xuất histogram màu sắc từ khung hình trong không gian màu HSV và LAB.
       Sử dụng 64 bins cho mỗi kênh màu để thu thập thông tin chi tiết hơn."""
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    hsv_hist = cv2.calcHist([hsv], [0, 1, 2], None, [bins]*3, [0, 180, 0, 256, 0, 256])
    lab_hist = cv2.calcHist([lab], [0, 1, 2], None, [bins]*3, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hsv_hist, hsv_hist)
    cv2.normalize(lab_hist, lab_hist)
    combined_hist = np.hstack((hsv_hist.flatten(), lab_hist.flatten()))
    return combined_hist


In [4]:
def extract_brightness_contrast_features(frame):
    """Trích xuất độ sáng và độ tương phản trung bình từ khung hình."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray)
    contrast = np.std(gray)
    return brightness, contrast

In [5]:
def extract_lbp_features(frame, radii=[1, 3, 8], n_points=[8, 16, 24], method='uniform'):
    """ Trích xuất đặc trưng Local Binary Patterns từ khung hình với nhiều bán kính và số điểm khác nhau. """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lbp_features = []
    for radius, points in zip(radii, n_points):
        lbp = local_binary_pattern(gray, points, radius, method)
        hist, _ = np.histogram(lbp.ravel(), bins=points+2, range=(0, points+2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)  # Chuẩn hóa histogram
        lbp_features.extend(hist)
    return np.array(lbp_features)


In [6]:
def extract_sift_features(frame, n_clusters=10):
    """Trích xuất và gom cụm đặc trưng SIFT từ khung hình, sử dụng K-means để giảm số lượng điểm đặc trưng."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    if descriptors is not None:
        if len(descriptors) > n_clusters:
            # Cập nhật n_init ở đây
            kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=0)
            kmeans.fit(descriptors)
            descriptors = kmeans.cluster_centers_
        else:
            # Trường hợp số lượng descriptors ít hơn số cụm yêu cầu, sử dụng chính các descriptors đó
            descriptors = np.mean(descriptors, axis=0, keepdims=True)
    else:
        # Nếu không có descriptors nào được tìm thấy, trả về None
        descriptors = None
    return descriptors


In [7]:
def process_video(video_path, sample_rate=30, bins=32, radii=[1, 3, 5], n_points=[8, 16, 24], sift_clusters=10):
    cap = cv2.VideoCapture(video_path)
    color_features = []
    texture_features = []
    sift_features = []
    brightness_features = []
    contrast_features = []
    frame_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_index % sample_rate == 0:
            color_hist = extract_color_histogram(frame, bins=bins)
            lbp_hist = extract_lbp_features(frame, radii=radii, n_points=n_points)
            sift_desc = extract_sift_features(frame, n_clusters=sift_clusters)
            brightness, contrast = extract_brightness_contrast_features(frame)

            if sift_desc is not None:
                sift_features.append(sift_desc.flatten())
            color_features.append(color_hist)
            texture_features.append(lbp_hist)
            brightness_features.append(brightness)
            contrast_features.append(contrast)

        frame_index += 1

    cap.release()
    # Aggregate and average features
    color_features = np.mean(np.array(color_features), axis=0) if color_features else None
    texture_features = np.mean(np.array(texture_features), axis=0) if texture_features else None
    sift_features = np.mean(np.array(sift_features), axis=0) if sift_features else None
    average_brightness = np.mean(brightness_features) if brightness_features else None
    average_contrast = np.mean(contrast_features) if contrast_features else None

    return color_features, texture_features, sift_features, average_brightness, average_contrast


In [8]:
def process_all_videos(directory_path, sample_rate=30, bins=32, radii=[1, 3, 5], n_points=[8, 16, 24], sift_clusters=10):
    data_directory = os.path.join('/kaggle/working/', 'data')
    os.makedirs(data_directory, exist_ok=True)

    features_database = {}

    for file_name in os.listdir(directory_path):
        if file_name.endswith('.mp4'):
            video_path = os.path.join(directory_path, file_name)
            color_features, texture_features, sift_features, average_brightness, average_contrast = process_video(video_path, sample_rate, bins, radii, n_points, sift_clusters)

            # Save the features as JSON
            save_features_as_json(color_features, os.path.join(data_directory, f'{file_name}_color_features.json'))
            save_features_as_json(texture_features, os.path.join(data_directory, f'{file_name}_texture_features.json'))
            save_features_as_json(sift_features, os.path.join(data_directory, f'{file_name}_sift_features.json'))
            save_features_as_json(average_brightness, os.path.join(data_directory, f'{file_name}_brightness.json'))
            save_features_as_json(average_contrast, os.path.join(data_directory, f'{file_name}_contrast.json'))

            # Store features in the database
            features_database[file_name] = (color_features, texture_features, sift_features, average_brightness, average_contrast)

            print(f"Processed and saved features for video {file_name} in JSON format")

    return features_database

In [9]:

def save_features_as_json(features, file_name):
    """Save features to a JSON file."""
    if isinstance(features, np.ndarray):
        features = features.tolist()  # Convert numpy arrays to lists
    elif isinstance(features, tuple):
        # Convert tuple of numpy arrays or numbers to list
        features = [f.tolist() if isinstance(f, np.ndarray) else f for f in features]

    with open(file_name, 'w') as f:
        json.dump(features, f, indent=4)

In [10]:
def calculate_distance(color_features, texture_features, sift_features, brightness, contrast, other_color, other_texture, other_sift, other_brightness, other_contrast, color_weight=0.25, texture_weight=0.15, sift_weight=0.4, brightness_weight=0.1, contrast_weight=0.1):
    color_dist = np.linalg.norm(color_features - other_color)
    texture_dist = np.linalg.norm(texture_features - other_texture)
    sift_dist = np.linalg.norm(sift_features - other_sift)
    brightness_dist = abs(brightness - other_brightness)
    contrast_dist = abs(contrast - other_contrast)

    weighted_dist = (color_weight * color_dist + texture_weight * texture_dist + sift_weight * sift_dist + brightness_weight * brightness_dist + contrast_weight * contrast_dist)
    return weighted_dist

In [11]:
directory_path = '/kaggle/input/data-2/data-2'
process_all_videos(directory_path, sample_rate=30)

Processed and saved features for video 019.mp4 in JSON format
Processed and saved features for video 017.mp4 in JSON format
Processed and saved features for video 026.mp4 in JSON format
Processed and saved features for video 030.mp4 in JSON format
Processed and saved features for video 009.mp4 in JSON format
Processed and saved features for video 028.mp4 in JSON format
Processed and saved features for video 015.mp4 in JSON format
Processed and saved features for video 013.mp4 in JSON format
Processed and saved features for video 012.mp4 in JSON format
Processed and saved features for video 004.mp4 in JSON format
Processed and saved features for video 003.mp4 in JSON format
Processed and saved features for video 020.mp4 in JSON format
Processed and saved features for video 025.mp4 in JSON format
Processed and saved features for video 002.mp4 in JSON format
Processed and saved features for video 018.mp4 in JSON format
Processed and saved features for video 008.mp4 in JSON format
Processe

{'019.mp4': (array([2.5410614e-05, 3.4015032e-04, 5.2029610e-04, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00], dtype=float32),
  array([0.03343364, 0.06616368, 0.03994117, 0.11156684, 0.20459346,
         0.1829485 , 0.07545042, 0.08077064, 0.10924286, 0.09588879,
         0.03653887, 0.03763262, 0.02676746, 0.0204022 , 0.01877025,
         0.02465278, 0.02994358, 0.04906443, 0.08577353, 0.07177903,
         0.04151765, 0.03023293, 0.02337288, 0.02328221, 0.02774595,
         0.03697   , 0.04994406, 0.36560957, 0.03735532, 0.02665316,
         0.02032793, 0.01481144, 0.01213638, 0.01068191, 0.01044801,
         0.01244068, 0.01367766, 0.0163561 , 0.0199783 , 0.02840471,
         0.04163098, 0.04111304, 0.02966001, 0.02474248, 0.01962529,
         0.016644  , 0.01309221, 0.01261333, 0.01246046, 0.01431809,
         0.01887201, 0.02433642, 0.04156973, 0.46605035]),
  array([29.087715, 21.37957 , 17.197754, ..., 14.785695, 11.959993,
         19.141533], dtype=float32),
  11